# Importing all the necessary libraries



In [4]:
pip install -U textblob

Note: you may need to restart the kernel to use updated packages.


In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')
stopword = stopwords.words('english')
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
from nltk.corpus import stopwords
import string
import json
from nltk.sentiment.util import *
from textblob import TextBlob

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sharanyakarthikeyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/sharanyakarthikeyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/sharanyakarthikeyan/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/sharanyakarthikeyan/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sharanyakarthikeyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Import and Read the data



In [8]:
with open('/Users/sharanyakarthikeyan/Downloads/tweets.json') as jfile:
  json_file = json.load(jfile)

# Convert into Data Frame 



In [9]:
df=pd.DataFrame(json_file).T


In [10]:
df.head(10)


,tweet_author,tweet_text
1374140386071961602,Hematopoiesis News,⚕️ Scientists conducted a Phase II study of ac...
1374032432173842437,"Michael Wang, MD",This phase 2 Acalabrutinib-Venetoclax (AV) tri...
1373902876553048065,1stOncology,#NICE backs #AstraZenecas #Calquence for #CLL ...
1373656782367813635,Toby Eyre,#acalabrutinib is a valuable option in pts int...
1372941634334232586,Lymphoma Hub,NICE has recommended the use of acalabrutinib ...
1372927482278539265,David Ledger,NICE backs AstraZeneca’s Calquence for CLL htt...
1372911723305394179,N Wales Cancer Forum,This is England for now - these decisions usua...
1372888121159868423,European Pharmaceutical Review,"AstraZeneca’s Calquence (acalabrutinib), a che..."
1372866915081797632,Graham Collins,Superstar @tobyeyre82 responding to the excell...
1372825553837944834,CLL Ireland,CLL patients all know the drug Ibrutinib and y...


# checking for null values and unique writters

print('total no of null valus in the data:\n',df.isnull().sum())
print('total no tweet author:',df.tweet_author.nunique())

In [11]:
def clean_data(df):
    df = df.lower()
    df = re.sub(r'[^(a-zA-Z)\s]','',df)
    #remove urls
    df = re.sub(r'http\S+', " ", df)
    #remove mentions
    df = re.sub(r'@\w+',' ',df)
    #remove hashtags
    df= re.sub(r'#\w+', ' ',df)
    #remove digits
    df = re.sub(r'\d+()', ' ', df)
    #remove html tags and umber
    df = re.sub('r<.*?>',' ', df)
    #remove stopwords
    df = df.split()
    df = " ".join([word for word in df if not word in stopword])
    return df
df['tweet_text']=df['tweet_text'].apply(lambda x:clean_data(x))
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

df['tweet_text']= df['tweet_text'].apply(lambda x: remove_punct(x))
#Tokenization of the text data
def tokenization(text):
    text = re.split('\W+', text)
    return text
df['tweet_text'] = df['tweet_text'].apply(lambda x: tokenization(x.lower()))


# objective 1
Get the most frequent entities from the tweets. and we convert them into csv

In [12]:
df1=df['tweet_text']

In [13]:
ps = nltk.PorterStemmer()
def stemming(text):
    text = [ps.stem(word) for word in text]
    return text
df1 = df1.apply(lambda x: stemming(x))

In [14]:
wn = nltk.WordNetLemmatizer()
def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text
df1= df1.apply(lambda x: lemmatizer(x))

# Stanford NLP NER



In [15]:
from nltk.tag.stanford import StanfordNERTagger
!wget 'https://nlp.stanford.edu/software/stanford-ner-2018-10-16.zip' -O stanford-ner-2018-10-16.zip
!unzip stanford-ner-2018-10-16.zip

nltk.download('punkt')



--2022-11-16 12:25:35--  https://nlp.stanford.edu/software/stanford-ner-2018-10-16.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://downloads.cs.stanford.edu/nlp/software/stanford-ner-2018-10-16.zip [following]
--2022-11-16 12:25:37--  https://downloads.cs.stanford.edu/nlp/software/stanford-ner-2018-10-16.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180358328 (172M) [application/zip]
Saving to: ‘stanford-ner-2018-10-16.zip’

stanford-ner-2018-1 100%[===================>] 172.00M  5.53MB/s    in 86s     

2022-11-16 12:27:06 (1.99 MB/s) - ‘stanford-ner-2018-10-16.zip’ saved [180358328/180358328]

Archive:  stanford-ner-2018-10-16.

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sharanyakarthikeyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
st = StanfordNERTagger('stanford-ner-2018-10-16/classifiers/english.all.3class.distsim.crf.ser.gz',
                       'stanford-ner-2018-10-16/stanford-ner.jar',
                       encoding='utf-8')

In [19]:
val = [item if isinstance(df1, str) else " ".join(item) for item in df1 ]
seen = set()
val = [x for x in val if x not in seen and not seen.add(x)]

In [20]:
classified_text = st.tag(val)


In [21]:
entity= pd.DataFrame(classified_text,columns=['Entity Name','Entity Type'])

#Here We Remove Entities_type column from data we does not requierd this but the help to recgonized the type of entity

all_entities = (entity.groupby(by=['Entity Name'])
                          .size()
                          .sort_values(ascending=False)
                          .reset_index().rename(columns={0 : 'Frequency'}))
all_entities.head(10)

,Entity Name,Frequency
0,acalabrutinib,1306
1,calquenc,893
2,patient,790
3,covid,694
4,astrazeneca,598
5,cll,562
6,trial,425
7,lymphocyt,388
8,chronic,351
9,leukemia,342


In [23]:
all_entities.to_csv('objective1.csv')